In [6]:
import os
import transformers, datasets
from huggingface_hub import notebook_login
notebook_login()

In [28]:
import pandas as pd
import torch
import torch.nn as nn

In [29]:
data_dir = './data/singapore_airlines_reviews.csv'

In [30]:
dataset = pd.read_csv(data_dir)

In [36]:
filtered_ds = dataset.drop(columns=['published_date', 'published_platform', 'type', 'helpful_votes','title'])
filtered_ds['label'] = filtered_ds['rating'].map(lambda x: 1 if x > 2 else 0)
filtered_ds = filtered_ds.drop(columns=['rating'])
filtered_ds['text'] = filtered_ds['text'].str.lower()

In [37]:
filtered_ds.head()

text  label
0  we used this airline to go from singapore to l...      1
1  the service on singapore airlines suites class...      1
2  booked, paid and received email confirmation f...      0
3  best airline in the world, seats, food, servic...      1
4  premium economy seating on singapore airlines ...      0

In [65]:
hf_ds = datasets.Dataset.from_pandas(filtered_ds).shuffle()

In [66]:
hf_len = len(hf_ds)
split = 0.8
train_ds, test_ds = datasets.Dataset.from_dict(hf_ds[:int(0.8*hf_len)]), datasets.Dataset.from_dict(hf_ds[int(0.8*hf_len):])

In [75]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_train_ds = train_ds.map(tokenize_function, batched=True)
tokenized_test_ds = test_ds.map(tokenize_function, batched=True)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [67]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-cased", num_labels=2)

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [72]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

In [76]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_ds,
    eval_dataset=tokenized_test_ds,
)

In [ ]:
trainer.train()